# Manipulating Time Series Data in Python

## Table of Contents

- [Introduction](#intro)
- [Working with Time Series in Pandas](#s1)


---
<a id='intro'></a>

## Introduction

In this course you'll learn the basics of manipulating time series data. Time series data are data that are indexed by a sequence of dates or times. You'll learn how to use methods built into Pandas to work with this index. You'll also learn how resample time series to change the frequency. This course will also show you how to calculate rolling and cumulative values for times series.

In [2]:
import pandas as pd

path = "data/dc27/"

---
<a id='s1'></a>

## Working with Time Series in Pandas

<img src="images/ts2_001.png" alt="" style="width: 400px;"/>

### Your first time series

Create a week of data, iterate over the result, and obtain the `dayofweek` and `weekday_name` for each date.

In [4]:
# Create the range of dates here
# Use pd.date_range to create seven dates starting from '2017-1-1' 
#  at (default) daily frequency. Use the arguments start and periods. 
#  Assign the result to seven_days.
seven_days = pd.date_range(start='2017-1-1', periods=7, freq='D')

seven_days

DatetimeIndex(['2017-01-01', '2017-01-02', '2017-01-03', '2017-01-04',
               '2017-01-05', '2017-01-06', '2017-01-07'],
              dtype='datetime64[ns]', freq='D')

In [11]:
# Iterate over the dates and print the number and name of the weekday
for day in seven_days:
    print(day.dayofweek, day.day_name())

6 Sunday
0 Monday
1 Tuesday
2 Wednesday
3 Thursday
4 Friday
5 Saturday


In [ ]:
---
<a id='s1'></a>

In [ ]:
<img src="images/ts2_001.png" alt="" style="width: 400px;"/>